# Decadal Calendar Fix

In [9]:
import xarray as xr
from rook.utils.decadal_fixes import apply_decadal_fixes, decadal_fix_calendar
from roocs_utils.xarray_utils.xarray_utils import open_xr_dataset
from rooki import rooki

import tempfile

In [15]:
tmp_dir = tempfile.TemporaryDirectory(dir=None)
tmp_dir.cleanup()

In [2]:
ds_id = "c3s-cmip6-decadal.DCPP.EC-Earth-Consortium.EC-Earth3.dcppA-hindcast.s1976-r3i1p1f1.Amon.psl.gr.v20201215"


## ds with mixed gregorian and proleptic gregorian



In [3]:
result = rooki.subset(ds_id, time="1984-11/1986-10", original_files=True)

result.ok

True

In [4]:
result.download_urls()

['https://data.mips.climate.copernicus.eu/thredds/fileServer/esg_c3s-cmip6/DCPP/EC-Earth-Consortium/EC-Earth3/dcppA-hindcast/s1976-r3i1p1f1/Amon/psl/gr/v20201215/psl_Amon_EC-Earth3_dcppA-hindcast_s1976-r3i1p1f1_gr_198411-198510.nc',
 'https://data.mips.climate.copernicus.eu/thredds/fileServer/esg_c3s-cmip6/DCPP/EC-Earth-Consortium/EC-Earth3/dcppA-hindcast/s1976-r3i1p1f1/Amon/psl/gr/v20201215/psl_Amon_EC-Earth3_dcppA-hindcast_s1976-r3i1p1f1_gr_198511-198610.nc']

In [5]:
dsets = [
    decadal_fix_calendar(ds_id, open_xr_dataset(file)) for file in result.download()
    # open_xr_dataset(file) for file in result.download()
]

In [6]:
for ds in dsets:
    print(ds.time.encoding.get("calendar"))


gregorian
standard


In [7]:
ds = xr.concat(dsets, dim="time")
ds.time.encoding.get("calendar")

'gregorian'

In [8]:
ds_mod = apply_decadal_fixes(ds_id, ds)
ds_mod

<xarray.Dataset>
Dimensions:      (time: 24, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time         (time) object 1984-11-16 00:00:00 ... 1986-10-16 12:00:00
  * lat          (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon          (lon) float64 0.0 0.7031 1.406 2.109 ... 357.9 358.6 359.3
    reftime      datetime64[ns] 1976-11-01
    leadtime     (time) float64 2.937e+03 2.967e+03 ... 3.606e+03 3.636e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (time, bnds) object 1984-11-01 00:00:00 ... 1986-11-01 00:00:00
    lat_bnds     (time, lat, bnds) float64 -90.0 -89.11 -89.11 ... 89.11 90.0
    lon_bnds     (time, lon, bnds) float64 -0.3516 0.3516 0.3516 ... 358.9 359.6
    psl          (time, lat, lon) float32 9.994e+04 9.994e+04 ... 1.014e+05
    realization  int32 3
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    nominal_resolution:          100 km
    sub_experiment:              initialized near end of year 1976
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s197611